In [23]:
import cv2
import math
import numpy as np
from  ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [24]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 2050


In [25]:
model_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\Yolo models\yolov8m.pt"
video_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\test_data\test_video\video 1.avi"

In [26]:
model = YOLO(model_path)

In [27]:
tracker = DeepSort(max_age=30)

In [28]:
cap = cv2.VideoCapture(video_path)

In [29]:
if not cap.isOpened():
    print("Error : Could not open video")
    exit()
else:
    print("Video is opened and working")

Video is opened and working


In [30]:
def obj_Detection(results, detections):
    for box in results.boxes:
        x1, y1, x2, y2 = map(int,box.xyxy[0])
        class_id = int(box.cls[0])
        class_name = results.names[class_id]
        confidence = float(box.conf[0])
        
        detections.append(([x1,y1,x2-x1,y2-y1], confidence, class_name))

In [31]:
stall_cars = []
def stall_movement_fn(car_coordinates, car_id):
    def eucledian(base_coord, pt):
        pt1 = (base_coord[0] - pt[0])**2
        pt2 = (base_coord[1] - pt[1])**2
        dist = math.sqrt(pt1+pt2)
        return dist

    coords = car_coordinates[car_id]
    stalled = True
    if(len(coords) > 10):
        recent_coords = coords[-10:]
        base_coord = recent_coords[0]
        for pt in recent_coords[1:]:
            distance = eucledian(base_coord, pt)
            if distance >= 5: #pixel_threshold
                stalled = False
                break
    else:
        stalled = False
        
    return stalled
    
    

In [32]:
def alert(car_id, reason):
    if(reason=="Opposite_direction"):
        print(f"{car_id} is moving in Opposite Direction.")
    elif(reason=="Stalled"):
        print(f"{car_id} is stalled.")
    else:
        print(f"{car_id} is ok")

In [33]:
opposite_cars = []
def detect_car_movement(car_coordinates, expected = [0,-1], threshold = 0.4):
    
    def normalize_vector(v):
        mag = math.sqrt(v[0]**2 + v[1]**2)
        if(mag!=0):
            return [v[0]/mag, v[1]/mag]
        else:
            return [0,0]
        
    def cosine_similarity(movment_vec, expected):
        dot_num = (movment_vec[0] * expected[0]) + (movment_vec[1] * expected[1])
        mag1 = math.sqrt((movment_vec[0]**2)+(movment_vec[1]**2))
        mag2 = math.sqrt((expected[0]**2)+(expected[1]**2))
        if(mag1 and mag2):
            return dot_num / (mag1 * mag2)
        else:
            return 1
    
    for car_id, coords in car_coordinates.items():
        
        if(len(coords)<2):
            continue
        
        x1, y1 = coords[-2] 
        x2, y2 = coords[-1]
        
        if(x2-x1 == 0 and y2-y1 == 0):
            if(stall_movement_fn(car_coordinates, car_id)):
                if(car_id not in stall_cars):
                    stall_cars.append(car_id)
                    alert(car_id,"Stalled")
        movment = [x2-x1, y2-y1]
        movement_unit = normalize_vector(movment)
        similarity = cosine_similarity(movement_unit, expected)
        
        
        if(similarity < -threshold):
            if(car_id not in opposite_cars):
                alert(car_id,"Opposite_direction")
                opposite_cars.append(car_id)
        

In [34]:
car_coordinates = {}
def car_coords_fn(track_id, x1, y1, x2, y2):
    center_x = int((x1 + x2)/2)    
    center_y = int((y1 + y2)/2)
    if track_id not in car_coordinates:
        car_coordinates[track_id] = []
    if len(car_coordinates[track_id]) >= 20:
        car_coordinates[track_id].pop(0)
    car_coordinates[track_id].append([center_x,center_y])
    detect_car_movement(car_coordinates)

In [35]:
def draw_Tracks(tracks,frame):
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        class_name = track.get_det_class()
        
        car_coords_fn(track_id, x1, y1, x2, y2)
    
        label = f"ID {track_id} | {class_name}"
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10,), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
        

In [36]:
while True:
   ret, frame = cap.read()
   if not ret:
       break
#    frame = bev_func(frame)
   results = model(frame, conf = 0.6)[0]
   detections = []
   
   obj_Detection(results, detections)
   
   tracks = tracker.update_tracks(detections, frame = frame)
   
   draw_Tracks(tracks,frame)
   
   cv2.imshow("Output", frame)
   if(cv2.waitKey(1) and 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
   
   


0: 480x640 7 cars, 49.8ms
Speed: 22.6ms preprocess, 49.8ms inference, 24.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 35.7ms
Speed: 3.3ms preprocess, 35.7ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 33.5ms
Speed: 3.0ms preprocess, 33.5ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 33.7ms
Speed: 4.0ms preprocess, 33.7ms inference, 7.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 30.8ms
Speed: 4.0ms preprocess, 30.8ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 36.0ms
Speed: 3.1ms preprocess, 36.0ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 30.6ms
Speed: 3.8ms preprocess, 30.6ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 cars, 31.8ms
Speed: 3.3ms preprocess, 31.8ms inference, 6.3ms postprocess per image at shape (1, 3, 480, 640)

0: 48

In [37]:
total_cars = len(car_coordinates)
print(f"Total Cars = {total_cars}")
print("--"*130)

for track_id, coords in car_coordinates.items():
    print(f"Car Id {track_id} -> {coords}")

Total Cars = 18
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Car Id 1 -> [[176, 120], [175, 118], [174, 116], [174, 114], [173, 112], [172, 110], [171, 108], [170, 106], [170, 104], [169, 102], [168, 100], [167, 98], [167, 96], [166, 94], [165, 92], [164, 90], [164, 88], [163, 86], [162, 84], [162, 82]]
Car Id 6 -> [[215, 139], [214, 138], [214, 137], [213, 136], [213, 134], [212, 133], [211, 132], [210, 131], [209, 130], [208, 129], [208, 127], [207, 126], [207, 125], [206, 123], [205, 122], [205, 121], [204, 120], [204, 119], [203, 118], [203, 117]]
Car Id 8 -> [[271, 143], [269, 141], [267, 140], [265, 138], [264, 136], [262, 135], [260, 133], [258, 131], [256, 130], [254, 128], [253, 126], [251, 125], [249, 123], [247, 121], [245, 120], [244, 118], [242, 116], [240, 115

In [38]:
print("\n---Stalled Vehicle---\n")
for s_car in stall_cars:
    print(f"Car Id {s_car} Stalled.")
print("\n\n---Opposite Vehicle---\n")
for op_car in opposite_cars:
    print(f"Car Id {op_car} is moving in opposite direction.")


---Stalled Vehicle---



---Opposite Vehicle---

Car Id 10 is moving in opposite direction.
